In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import dataloader
from torchvision.transforms import transforms

from mnist.digit_model import Digit
from mnist.mnist_dataset import MNISTDataset

In [2]:
pipeline = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # 正则化
])

trainDataset = MNISTDataset(
    'MNIST_data/',
    'train-images.idx3-ubyte',
    'train-labels.idx1-ubyte',
    transform=pipeline)
testDataset = MNISTDataset(
    'MNIST_data/',
    "t10k-images.idx3-ubyte",
    "t10k-labels.idx1-ubyte",
    transform=pipeline)

train_loader = dataloader.DataLoader(
    dataset=trainDataset,
    batch_size=100,
    shuffle=False)
test_loader = dataloader.DataLoader(
    dataset=testDataset,
    batch_size=100,
    shuffle=False)

In [3]:
#定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    # 训练模型
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 部署到device
        data, target = data.to(device), target.to(device)
        # 梯度初始化为零
        optimizer.zero_grad()
        # 训练后的结果
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)
        # 反向传播
        loss.backward()
        # 参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train -- Epoch: {}\t Loss: {:.6f}".format(epoch, loss.item()))

In [4]:
# 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 准确率
    correct = 0.0
    # 测试损失
    test_loss = 0.0
    with torch.no_grad():  # 不会计算梯度，也不进行反向传播
        for data, target in test_loader:
            # 部署到device
            data, target = data.to(device), target.to(device)
            # 测试部署
            output = model(data)
            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.argmax(dim=1)
            # 累计准确率
            correct += pred.eq(target.view_as(pred)).sum().item()
        
        test_loss /= len(test_loader.dataset)
        acc = 100.0 * correct / len(test_loader.dataset)
        print("Test -- Average loss: {:.4f}\t Accuracy: {:.3f}".format(test_loss, acc))
        
    return acc

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

In [5]:
EPOCHES = 10
LR = 0.001
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))
# net = Digit().to(DEVICE)

# 定义优化器，设置学习率和权重衰减
optimizer = optim.SGD(net.parameters(), lr=LR, weight_decay=0.9)

accuracy = []
for epoch in range(1, EPOCHES + 1):
    # 训练模型
    train_model(net, DEVICE, train_loader, optimizer, epoch)
    # 测试并记录精度
    accuracy.append(test_model(net, DEVICE, test_loader))

D:\anaconda3\envs\jupytertest\lib\site-packages\torchvision\transforms\functional.py:126: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\Users\builder\tkoch\workspace\pytorch\pytorch_1647970138273\work\torch\csrc\utils\tensor_numpy.cpp:189.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


Train -- Epoch: 1	 Loss: 2.295365
Test -- Average loss: 0.0007	 Accuracy: 97.700
Runtime: 45.8007s

Train -- Epoch: 2	 Loss: 0.070032
Test -- Average loss: 0.0004	 Accuracy: 98.530
Runtime: 34.7990s

Train -- Epoch: 3	 Loss: 0.053770
Test -- Average loss: 0.0004	 Accuracy: 98.740
Runtime: 34.4970s

Train -- Epoch: 4	 Loss: 0.046258


KeyboardInterrupt: 

In [ ]:
epoches = [i for i in range(1, EPOCHES + 1)]

plt.plot(epoches, accuracy)
plt.title('accuracy')
plt.show()
